<a href="https://colab.research.google.com/github/aadhamashraf/CareerMatch-AI/blob/main/tree/main/ai-features/cv_parsing_section_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber docx2txt transformers torch scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.8 MB/s eta 0:00:00


In [2]:
import pdfplumber
import docx2txt

def extract_text_from_file(file_path):
    if file_path.endswith(".pdf"):
        text = ""
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    elif file_path.endswith(".docx"):
        text = docx2txt.process(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload PDF or DOCX.")
    return text

In [3]:
import re

def split_text_into_sentences(text):
    sentences = re.split(r'\n+|(?<=[.!?])\s+', text.strip())
    return [s.strip() for s in sentences if len(s.strip()) > 10]

In [4]:
# from transformers import pipeline

# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# labels = ["Education", "Experience", "Projects", "Skills", "Certifications"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [5]:
# def classify_cv_text(sentences):
#     results = []
#     for sentence in sentences:
#         pred = classifier(sentence, labels)
#         label = pred["labels"][0]
#         score = pred["scores"][0]
#         results.append({"text": sentence, "category": label, "confidence": round(score, 3)})
#     return results

In [7]:
file_path = "/content/Adham_Ashraf_Eltholth.pdf"
text = extract_text_from_file(file_path)
sentences = split_text_into_sentences(text)

['Adham Ashraf Eltholth',
 '+20 15 50840224 | adham.eltholth@gmail.com | October Gardens, Giza',
 'linkedin.com/in/adham-ashraf | github.com/aadhamashraf',
 'A multi-passionate computer science and AI undergraduate with a strong foundation in software engineering, machine',
 'learning, and intelligent systems.',
 'I aim to bridge the gap between innovation and implementation through meaningful,',
 'scalable, and impactful tech solutions.',
 'Passionate about AI explainability, full-stack development, and solving real-world',
 'problems with creative and detail-oriented thinking.',
 'Actively seeking opportunities to contribute to cutting-edge projects,',
 'grow through mentorship, and build futuristic, human-centric technology platforms.',
 '• Zewail City of Science, Technology and Innovation Aug.',
 '2022 – Sept.',
 'Bachelor of Science in Computer Science and Artificial Intelligence',
 '• Data Science and AI Intern July 2025 - Present',
 'Orange Innovation Egypt',
 'Worked on the dev

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd


df = pd.read_csv("/content/technical_cv_augmented_ollama.csv")

label2id = {label: idx for idx, label in enumerate(df["label"].unique())}
id2label = {idx: label for label, idx in label2id.items()}

df["label"] = df["label"].map(label2id)

dataset_hf = Dataset.from_pandas(df)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_fn(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

dataset_hf = dataset_hf.map(tokenize_fn, batched=True)
dataset_hf = dataset_hf.rename_column("label", "labels")
dataset_hf.set_format("torch")

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

args = TrainingArguments(
    output_dir="cv-section-classifier",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    do_eval=True,   # old-style flag to enable evaluation
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_hf,
    eval_dataset=dataset_hf,
)

trainer.train()

Map:   0%|          | 0/3896 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
